## Prueba de funciones

📥Importaciones 

In [1]:

# Importación del módulo gc para la recolección de basura
import gc

# Importación del módulo pickle para la serialización de objetos
import pickle

# Importación del módulo warnings para controlar advertencias
import warnings

# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd

# Importación de funciones desde data_utils
from data_utils import analisis_sentimiento, ej_review_sentimiento



# Configuración de la omisión de advertencias
warnings.filterwarnings("ignore")

# Carga de la extensión autoreload y configuración para recargar automáticamente módulos
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


📦 Extracción de los conjuntos de datos
* ✍️ **Extraemos df_reviews:** usuarios que realizaron reseñas de los juegos que consumen.
* 🎮 **Extraemos df_games:** juegos disponibles en la plataforma.
* 📊 **Extraemos df_items:** consumo de los juegos por parte de los usuarios.


In [ ]:
df_steam_games =   pd.read_parquet('../0 Dataset/steam_games_LISTO.parquet')
df_user_items =   pd.read_parquet('../0 Dataset/user_items_LISTO.parquet')
df_user_reviews = pd.read_parquet('../0 Dataset/user_review_LISTO.parquet')


##### 1 🌐 `developer(desarrollador: str)`:

Devuelve la cantidad de juegos y porcentaje de contenido Free por año según empresa desarrolladora

 **📂 steam_games.json**: developer, release_date, price

🌐Para acelerar la velocidad de la api, he decidido crear un DF especifico. 

In [ ]:
df_API_developer = df_steam_games[["developer","user_id", "release_year", "price"]]
df_API_developer.to_parquet("../0 Dataset/df_API_developer.parquet")
def developer(desarrollador: str):
    df_dev = df_API_developer[df_API_developer['developer'] == desarrollador]

    grouped = df_dev.groupby('release_year').agg(
        items=('user_id', 'count'),  # Cambiar de (x == 0).sum() a 'count' o 'sum'
        gratis=('price', lambda x: (x == 0).sum())
    )

    result = []
    for year, row in grouped.iterrows():
        juegos = int(row["items"])
        gratis_percent = round(row["gratis"] / juegos * 100, 2) if juegos > 0 else 0
        result.append({
            "Año": int(year),
            "Juegos": juegos,
            "Gratis %": gratis_percent
        })
    
    # Llamamos al recolector de basura
    gc.collect()

    return result
# Example usage
resultado = developer('Valve')
print(resultado)

##### 🌐 userdata(User_id: str) : 
Dinero gastado por el usuario, porcentaje de recomendación y cantidad de items.

1. 📂 **user_reviews.json**: user_id, recommend, helpful.total
2. 📂 **users_items.json**: user_id, items_count

In [ ]:

# Unir los dataframes en base a la columna "user_id"
df_userdata = pd.merge(df_user_reviews[['user_id', 'reviews_recommend', 'reviews_helpful']], df_user_items[['user_id', 'items_count']], on='user_id')

df_userdata
def userdata(user_id):
    user = df_API_userdata[df_API_userdata['user_id'] == user_id]
    if len(user) > 0:
        user_dict = {
            'dinero_gastado': user['items'].sum(),
            'porcentaje_recomendacion': user['recommend'].mean(),
            'cantidad_items': len(user)
        }
        return user_dict
    else:
        return None
user = userdata('evcentric')
if user is not None:
    print(user)



"""
_______________________________
1 DEF userdata(User_id: str):  

devolver cantidad de dinero gastado por el usuario
el porcentaje de recomendación en base a reviews.recommend 
y cantidad de items. 
_______________________________
"""
import pandas as pd
from typing import Dict

def userdata(User_id: str) -> Dict[str, float]:
    """
    Devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y la cantidad de items.
    
    Parameters:
    User_id (str): Identificador único del usuario.
    
    Returns:
    Dict[str, float]: Diccionario con los datos del usuario (dinero gastado, porcentaje de recomendación, cantidad de items).
    """
    # Leer los archivos Parquet
    user_reviews_df = pd.read_parquet('user_reviews.parquet')
    users_items_df = pd.read_parquet('users_items.parquet')
    
    # Obtener los datos del usuario
    user_data = users_items_df[users_items_df['user_id'] == User_id].iloc[0]
    
    # Obtener las revisiones del usuario
    user_reviews = user_reviews_df[user_reviews_df['user_id'] == User_id]
    
    # Calcular el total de dinero gastado y el porcentaje de recomendación
    total_spent = user_reviews['helpful.total'].sum()
    recommend_count = len(user_reviews[user_reviews['recommend']])
    total_reviews = len(user_reviews)
    recommend_percentage = (recommend_count / total_reviews) * 100 if total_reviews > 0 else 0.0
    
    return {
        "Dinero gastado": total_spent,
        "% de recomendación": recommend_percentage,
        "cantidad de items": user_data['items_count']
    }



### 2_ 📊 Análisis de Sentimientos

**'sentiment_analysis'** reemplaza a **'reviews_review'** : se crea esta nueva columna para realizar un análisis de sentimientos en los comentarios de los usuarios.

**Utilizamos la biblioteca TextBlob** , que es una herramienta de procesamiento de lenguaje natural (NLP) en Python en esta columna y aplico un análisis de sentimiento básico.

La escala utilizada es la siguiente:

* 👎 **0** si el sentimiento es **malo.**
* 😐 **1** si el sentimiento es **neutral o no hay revisión.**
* 👍 **2** si el sentimiento es **positivo.**

Esta metodología asigna un valor numérico a cada texto, en este caso, a los comentarios de los usuarios en relación con un juego específico, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo.

El análisis se basa en la polaridad calculada por TextBlob, donde se considera que las polaridades negativas están por debajo de -0.2, las positivas por encima de 0.2 y las neutrales entre estos valores.


In [ ]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['reviews_review'].apply(analisis_sentimiento)
df_user_reviews.head()
#Reviso algunos ejemplos para cada una de las clases de sentimiento.
ej_review_sentimiento(df_user_reviews['reviews_review'], df_user_reviews['sentiment_analysis'])
🗑️ Al final del proceso, la columna 'reviews_review' se elimina del conjunto de datos. 
df_user_reviews = df_user_reviews.drop(columns=['reviews_review'])
df_user_reviews.columns
df_user_reviews.info()


## Machine Learning item x item
### def **recomendacion_juego( *`id de producto`* )**:
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado
falta similaridad indices .pkl

In [ ]:

df_ML = pd.read_parquet("./0 Dataset/F_df_funciones.parquet")

def recomendacion_juego(item_id: int):
    with open("./0 Dataset/similaridad_indices.pkl", "rb") as archivo:
        pickle.load(archivo)
    
    try:
        with open('./0 Dataset/similaridad_indices.pkl', 'rb') as f:
            similaridad_indices = pickle.load(f)
    except FileNotFoundError:
        matrix_sparse = csr_matrix(pd.crosstab(df_ML['item_id'], df_ML['user_id'], values=df_ML['reviews_recommend'], aggfunc='sum').fillna(0))
        simil = cosine_similarity(matrix_sparse)
        similaridad_indices = {item_id: simil[item_index].argsort()[-6:-1][::-1] for item_index, item_id in enumerate(df_ML['item_id'].unique().tolist())}
        with open('./0 Dataset/similaridad_indices.pkl', 'wb') as f:
            pickle.dump(similaridad_indices, f)
        del matrix_sparse, simil
    
    try:
        item_similar = [df_ML['item_id'].unique().tolist()[i] for i in similaridad_indices[item_id]]
        item_similar_nombre = [(df_ML[df_ML['item_id'] == item]['item_id'].values[0], df_ML[df_ML['item_id'] == item]['item_name'].values[0]) for item in item_similar]
        return {"Recomendaciones para Item ID": item_id, "Juegos Recomendados": item_similar_nombre}
    except KeyError:
        return {"error": f"Item ID {item_id} not found."}
    finally:
        # Liberar memoria utilizando el recolector de basura
        gc.collect()

recomendacion_juego(500)